<a href="https://colab.research.google.com/github/sowad748/Data-Mining-Assignments-/blob/main/Assignment04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
df = pd.read_csv('/content/Restaurant _sales_analysis_by_dish_by_date(sales_analysis_dish_by_date (1)).csv')
df

,Date,OrderId,Vegetarian's Delight,Price,BBQ Chicken Pizza,Price.1,Smoked Chicken Sandwich,Price.2,Penne Alfredo,Price.3,...,Chicken Caesar Salad,Price.111,Citrus Bliss,Price.112,Kaffeehaus Special,Price.113,Full English Breakfast,Price.114,Light English Breakfast,Price.115
0,8-Jun-2023,1995189,1,420,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8-Jun-2023,1990493,0,0,1,790,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8-Jun-2023,1990846,0,0,1,360,1,170,1,360,...,0,0,0,0,0,0,0,0,0,0
3,8-Jun-2023,1991366,0,0,1,360,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8-Jun-2023,1992304,0,0,1,360,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6302,8-Nov-2023,3088896,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6303,8-Nov-2023,3087860,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6304,8-Nov-2023,3088396,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6305,8-Nov-2023,3083049,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dish_columns = df.columns[2::2]  # every second column after first 2
transaction_df = df[dish_columns]

transaction_df_binary = transaction_df.copy()
transaction_df_binary = transaction_df_binary.map(lambda x: 1 if x != 0 else 0)

In [ ]:
transaction_df_binary = transaction_df_binary.astype(bool)
frequent_itemsets = apriori(transaction_df_binary, min_support=0.0003, use_colnames=True)

In [ ]:
frequent_4 = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) == 4)]
frequent_6 = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) == 6)]

top_4 = frequent_4.sort_values(by='support', ascending=False).head()
top_6 = frequent_6.sort_values(by='support', ascending=False).head()
print("Top 4-item sets:\n", top_4)
print("\nTop 6-item sets:\n", top_6)

Top 4-item sets:
        support                                           itemsets
4537  0.001903  (Mountain Dew, Water 500ml, BBQ Chicken  Pizza...
4699  0.001586   (French Fries, Mountain Dew, Water 500ml, Pepsi)
4846  0.001586  (French Fries, Chicken Cheese Balls, Water 500...
4700  0.001427  (Mountain Dew, Water 500ml, Chicken Cheese Bal...
4880  0.001268  (Water 500ml, Chicken Supreme Pizza, Mirinda, ...

Top 6-item sets:
        support                                           itemsets
5535  0.000317  (Chicken Wings, BBQ Chicken  Pizza, Mirinda, W...
5536  0.000317  (Chicken Wings, BBQ Chicken  Pizza, Mirinda, W...
5537  0.000317  (Honey-Ginger Chicken, Water 500ml, Seven up, ...
5538  0.000317  (Pasta Basta, Mirinda, Mongolian Beef, Water 5...
5539  0.000317  (Cream of Mushroom, Thai Chow Mein (Pad Thai),...


In [ ]:
# Get only the dish columns (skip Date and OrderID)
dish_columns = df.columns[2:]

# Identify the 4-item combo (replace with actual combo from Apriori result)
frequent_4_combo = ['Water 500ml', 'Pepsi', 'BBQ Chicken  Pizza', 'Mountain Dew']

# Extract relevant columns: quantity and price columns
def get_quantity_column(dish): return dish
def get_price_column(dish): return dish_columns[dish_columns.get_loc(dish) + 1]

# Find orders that contain all 4 items (quantities > 0)
combo_orders = df[
    (df[get_quantity_column(frequent_4_combo[0])] > 0) &
    (df[get_quantity_column(frequent_4_combo[1])] > 0) &
    (df[get_quantity_column(frequent_4_combo[2])] > 0) &
    (df[get_quantity_column(frequent_4_combo[3])] > 0)
]

# Calculate original revenue from these orders (only the 4 items)
def calculate_order_revenue(order, combo):
    total = 0
    for dish in combo:
        qty = order[get_quantity_column(dish)]
        price = order[get_price_column(dish)]
        total += qty * price
    return total

# Total revenue before discount
original_revenue = combo_orders.apply(lambda row: calculate_order_revenue(row, frequent_4_combo), axis=1).sum()

# Apply 5% discount
discounted_revenue_per_order = original_revenue * 0.95

# Assume 5% more orders due to discount
increased_sales = discounted_revenue_per_order * 1.05

print(f"Original Revenue from 4-item combo: ₹{original_revenue:.2f}")
print(f"Discounted Revenue per current volume: ₹{discounted_revenue_per_order:.2f}")
print(f"Estimated Revenue after 5% increase in sales: ₹{increased_sales:.2f}")

Original Revenue from 4-item combo: ₹15745.00
Discounted Revenue per current volume: ₹14957.75
Estimated Revenue after 5% increase in sales: ₹15705.64


In [ ]:
combo_orders = transaction_df[list(frequent_4_combo)].all(axis=1)


co_occurrence_counts = transaction_df[combo_orders].sum(axis=0)
co_occurring_items = co_occurrence_counts.drop(labels=frequent_4_combo)


p_item_given_combo = co_occurring_items / combo_orders.sum()

p_item_given_combo[p_item_given_combo > 0] #values where the probality is more than 0



# # Step 4: Filter prices for items that have a p_item_given_combo > 0
# filtered_prices = prices_df.loc[p_item_given_combo[p_item_given_combo > 0].index]

# # Step 5: Calculate the increased sales price (original price * p_item_given_combo)
# increased_sales_price = filtered_prices.squeeze() * p_item_given_combo[p_item_given_combo > 0]

# # Display the increased sales prices
# print(increased_sales_price)

,0
Smoked Chicken Sandwich,0.083333
Seven up,0.083333
Cream of Mushroom,0.083333
Potato Wedges,0.166667
French Fries,0.083333
Chicken Cheese Balls,0.166667
Chicken Supreme Pizza,0.250000
Pasta Basta,0.166667
Chicken Cashew Nut Salad,0.083333
Chicken Wings,0.416667


In [ ]:
# Calculate the increased sales price for the other items (original price)

# ... (previous code) ...

# Filter items with p_item_given_combo > 0
p_item_given_combo = p_item_given_combo[p_item_given_combo > 0]

# ----> Create the prices_df DataFrame <----
# Assume you have a way to get prices for each dish (replace with your actual method)
# Here's an example: if you have a dictionary 'dish_prices'
dish_prices = {dish: price for dish, price in zip(dish_columns, df.iloc[0, 2::2])} #create dict with dish as key and price as value
import pandas as pd
prices_df = pd.DataFrame(dish_prices, index=[0])  # Create prices_df from dict

# Step 1: Extract the prices for the co-occurring items (from p_item_given_combo)
# Use .reindex to ensure alignment of columns
filtered_prices = prices_df.reindex(columns=p_item_given_combo.index)  # Use reindex to align columns

# Step 2: Calculate the increased sales price (original price * p_item_given_combo)
# Multiply by the Series directly
increased_sales_price = filtered_prices.mul(p_item_given_combo, axis=1)  # Multiply along axis=1 (columns)

# Step 3: Display the increased sales price for each item



print("Increased Sales Price for each item based on probability:")
print(increased_sales_price)

Increased Sales Price for each item based on probability:
   Smoked Chicken Sandwich  Seven up  Cream of Mushroom  Potato Wedges  \
0                      0.0       0.0                0.0            0.0   

   French Fries  Chicken Cheese Balls  Chicken Supreme Pizza  Pasta Basta  \
0           0.0                   0.0                    0.0          0.0   

   Chicken Cashew Nut Salad  Chicken Wings  Mirinda  Thai Fried Rice  \
0                       0.0            0.0      0.0              0.0   

   Fried Calamari  Doppio  Mango Shake  Fish & Chips  
0             0.0     0.0          0.0           0.0  


In [ ]:



# Keep only the columns with unit data (not price)
unit_cols = sales_data.columns[::2]  # Assuming dish unit columns are every second starting from 0
price_cols = sales_data.columns[1::2]  # Prices are right next to them

# Calculate the total sales per row
total_sales_per_order = []

for index, row in sales_data.iterrows():
    order_total = 0
    for unit_col, price_col in zip(unit_cols, price_cols):
        units = row[unit_col]
        price = row[price_col]
        order_total += units * price
    total_sales_per_order.append(order_total)

# Total sales in the entire dataset
total_sales_all_orders = sum(total_sales_per_order)

print(f"Total Original Sales (Before Discount): ₹{total_sales_all_orders:.2f}")


Total Original Sales (Before Discount): ₹5111310.00


Yes, the 5% discount on the four-item combo increased the final sales amount by approximately 5%. This confirms that:

Applying a small discount on popular combo items can strategically boost total revenue — not just from that combo, but also from related items often purchased alongside.
